In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sgp4.api import Satrec
from sgp4.conveniences import sat_epoch_datetime
import numpy as np
import gc
import glob
import os
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import MinMaxScaler
from scipy.optimize import curve_fit, root_scalar
import shutil

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, TimeDistributed, Dense
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

#### Pre-Processing

In [ ]:
decayed_df = pd.read_csv("data/tip.csv")
decayed_df['MSG_EPOCH'] = pd.to_datetime(decayed_df['MSG_EPOCH'], format='%Y-%m-%d %H:%M:%S')
decayed_df['DECAY_EPOCH'] = pd.to_datetime(decayed_df['DECAY_EPOCH'], format='%Y-%m-%d %H:%M:%S')
decayed_df = decayed_df.sort_values(by='MSG_EPOCH', ascending=False)
decayed_df = decayed_df.drop_duplicates(subset='NORAD_CAT_ID', keep='first')
decayed_df = decayed_df.sort_values(by='NORAD_CAT_ID').reset_index(drop=True)
decayed_df.head()

In [ ]:
decayed_df.info()

In [ ]:
decayed_list = decayed_df['NORAD_CAT_ID'].to_list()
decayed_tip = decayed_df['DECAY_EPOCH'].to_list()
len(decayed_list)

#### Exploratory Data Analysis

In [ ]:
decayed_df['RCS_SIZE'].value_counts().sort_index().plot(kind='bar', color='skyblue', edgecolor='black')

# Formatting
plt.title('Distribution of RCS_SIZE')
plt.xlabel('RCS_SIZE Category')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
def read_tle_file(tle_path):
    tles = []
    with open(tle_path) as f:
        lines = [l.strip() for l in f.readlines() if l.strip()]
    for i in range(0, len(lines), 2):
        # name = lines[i]
        line1 = lines[i]
        line2 = lines[i+1]
        tles.append((line1, line2))
    return tles

In [ ]:
def generate_df(tles):
    df = pd.DataFrame(columns=['EPOCH_DATE_TIME', 'ECC', 'B*', 'perigee', 'a'])
    for s, t in tles:
        satellite = Satrec.twoline2rv(s, t)
        df.loc[len(df)] = [sat_epoch_datetime(satellite), satellite.ecco, satellite.bstar, satellite.altp * satellite.radiusearthkm, satellite.a * satellite.radiusearthkm]
    df.set_index('EPOCH_DATE_TIME', inplace=True)
    df.sort_index(inplace=True)
    # df.sort_values('EPOCH_DATE_TIME', inplace=True)
    return df

def generate_df_altitude(tles):
    df = pd.DataFrame(columns=['EPOCH_DATE_TIME', 'altitude'])
    for s, t in tles:
        satellite = Satrec.twoline2rv(s, t)
        df.loc[len(df)] = [sat_epoch_datetime(satellite), satellite.altp * satellite.radiusearthkm]
    df.set_index('EPOCH_DATE_TIME', inplace=True)
    df.sort_index(inplace=True)
    df = df[df['altitude'] <= 200]
    if len(df) != 0:
        df = df[df['altitude'].idxmax() : ]
        df['hours'] = (df.index - df.index[0]).total_seconds() / 3600
    return df

def generate_df_altitude_with_tip(tles, time, alt=10):
    df = pd.DataFrame(columns=['EPOCH_DATE_TIME', 'altitude'])
    for idx, (s, t) in enumerate(tles):
        satellite = Satrec.twoline2rv(s, t)
        df.loc[len(df)] = [sat_epoch_datetime(satellite), satellite.altp * satellite.radiusearthkm]
    df.loc[len(df)] = [time.to_pydatetime() , alt]
    df['EPOCH_DATE_TIME'] = pd.to_datetime(df['EPOCH_DATE_TIME'], utc=True)
    df.sort_values(by=['EPOCH_DATE_TIME'])
    df = df[df['altitude'] <= 250]
    return df

In [ ]:
bstar = []
ecc = []
missedFiles = []
for id in decayed_list:
    try:
        tles = read_tle_file(f'objects/{id}.txt')
        df = generate_df(tles)
        trimmed = df[df['perigee'] <= 200]
        if len(trimmed) != 0:
            trimmed = trimmed[trimmed['perigee'].idxmax() : ]
            if len(trimmed) != 0:
                bstar.extend(trimmed['B*'].to_list())
                ecc.extend(trimmed['ECC'].to_list())
    except:
        missedFiles.append(id)

In [ ]:
ecc_series = pd.Series(ecc, name='Eccentricity')
bstar_series = pd.Series(bstar, name='B*')

# Plot Eccentricity
plt.figure(figsize=(10, 4))
ecc_series.plot(kind='hist', bins=50, color='orange', edgecolor='black')
plt.title('Distribution of Eccentricity (<=200km Perigee)')
plt.xlabel('Eccentricity')
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot B*
plt.figure(figsize=(10, 4))
bstar_series.plot(kind='hist',bins=50, color='red', edgecolor='black')
plt.title('Distribution of B* (<=200km Perigee)')
plt.xlabel('B*')
plt.grid(True)
plt.tight_layout()
plt.show()


#### Curve Fitting

In [ ]:
valid_decayed_list = []
valid_decayed_tips = []
missing_list = []
for id, tip in zip(decayed_list, decayed_tip):
    try:
        with open(f'objects/{id}.txt') as f:
            lines = [l.strip() for l in f.readlines() if l.strip()]
            valid_decayed_list.append(id)
            valid_decayed_tips.append(tip)
    except:
        missing_list.append(id)

In [ ]:
max_k = 5
thresh = 3
al = df['altitude'].iloc[:-1]
result = seasonal_decompose(al, model='additive',extrapolate_trend='freq', period=3)
residuals = pd.Series(result.resid)
mean = residuals.mean()
std = residuals.std()
threshold_upper = mean + thresh * std  # 3 standard deviations above
threshold_lower = mean - thresh * std  # 3 standard deviations below
major_shifts = residuals[(residuals > threshold_upper) | (residuals < threshold_lower)]

indexes = list(filter(lambda x: x != df.index[-2], major_shifts.index))
if len(indexes) != 0:
    tdf = df.drop(index=indexes)
else:
    tdf = df.copy()
tdf['time_hours'] = (tdf['EPOCH_DATE_TIME'] - tdf['EPOCH_DATE_TIME'].min()).dt.total_seconds() / 3600

tref = tdf['time_hours'].iloc[-1]

def model(t, *params):
    a1 = params[0]
    result = a1
    for k in range(2, max_k + 2):
        result += params[k - 1] * (tref-t) ** (1 / k)
    return result

t_data = tdf['time_hours'].values
y_data = tdf['altitude'].values

initial_guess = np.ones(max_k + 1)
    
params, covariance = curve_fit(model, t_data, y_data, p0=initial_guess)
    
# a1_opt, *ak_opt = params
# step = 1/60
# num = int(np.floor((t_data.max() - t_data.min()) / step)) + 1
# t_fit = np.linspace(t_data.min(), t_data.max(), num)
# y_fit = model(t_fit, *params)

def predict_times_for_altitudes(h_targets, params, t_bounds):
    predicted_times = []
    for h_target in h_targets:
        def func(t):
            return model(t, *params) - h_target

        f_a = func(t_bounds[0])
        f_b = func(t_bounds[1])

        if f_a * f_b > 0:
            # No sign change: root not guaranteed in the interval
            predicted_times.append(np.nan)
            continue

        try:
            result = root_scalar(func, bracket=t_bounds, method='toms748')
            if result.converged:
                predicted_times.append(result.root)
            else:
                predicted_times.append(np.nan)
        except ValueError:
            predicted_times.append(np.nan)
    return predicted_times

target_altitudes = np.linspace(200, 10, 25)
t_bounds = (t_data[0], t_data[-1])
times_to_targets = predict_times_for_altitudes(target_altitudes, params, t_bounds)
times_to_targets = np.array(times_to_targets)

if np.count_nonzero(np.isnan(times_to_targets)) == 1 and np.isnan(times_to_targets[-1]):
    times_to_targets[-1] = tdf['time_hours'].iloc[-1]

plt.figure(figsize=(8, 5))
fdf = tdf[tdf['altitude'] <= 200]
plt.scatter(fdf['time_hours'], fdf['altitude'],
            color='blue', label='Data points')

# plt.plot(t_fit, y_fit, color='red', linestyle='-', marker='o',
#          label=f'fit (degree={max_k})')


plt.plot(times_to_targets, target_altitudes, color='green', linestyle='--', marker='o',
         label=f'fit')

plt.xlabel('hours')
plt.ylabel('altitude')
plt.title('Fit: Epoch vs. Mean Perigee')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
for id in valid_decayed_list:
    source = f'objects/{id}.txt'
    destination = f'objects_dec/{id}.txt'
    shutil.copy2(source, destination)

In [ ]:
def generate_dataset(max_k=5):
    z = 0
    tip_alt = 10
    invalid_dataset = []
    for id, tip in zip(valid_decayed_list[z:], valid_decayed_tips[z:]):
        try:
            tles = read_tle_file(f'objects/{id}.txt')
            df = pd.DataFrame(columns=['EPOCH_DATE_TIME', 'altitude'])
            for idx, (s, t) in enumerate(tles):
                satellite = Satrec.twoline2rv(s, t)
                df.loc[len(df)] = [sat_epoch_datetime(satellite), satellite.altp * satellite.radiusearthkm]
            df.loc[len(df)] = [tip.to_pydatetime() , tip_alt]
            df['EPOCH_DATE_TIME'] = pd.to_datetime(df['EPOCH_DATE_TIME'], utc=True)
            df.sort_values(by=['EPOCH_DATE_TIME'],inplace=True)
            df = df[df['altitude'] <= 250]

            if len(df) < 10:
                invalid_dataset.append(id)
                continue
                
            df['epoch_ts'] = df['EPOCH_DATE_TIME'].apply(lambda dt: dt.timestamp())

            thresh = 3
            al = df['altitude'].iloc[:-1]
            result = seasonal_decompose(al, model='additive',extrapolate_trend='freq', period=3)
            residuals = pd.Series(result.resid)
            mean = residuals.mean()
            std = residuals.std()
            threshold_upper = mean + thresh * std  # 3 standard deviations above
            threshold_lower = mean - thresh * std  # 3 standard deviations below
            major_shifts = residuals[(residuals > threshold_upper) | (residuals < threshold_lower)]
            indexes = list(filter(lambda x: x != df.index[-2], major_shifts.index))
            if len(indexes) != 0:
                tdf = df.drop(index=indexes)
            else:
                tdf = df.copy()
            tdf['time_hours'] = (tdf['EPOCH_DATE_TIME'] - tdf['EPOCH_DATE_TIME'].min()).dt.total_seconds() / 3600
            tref = tdf['time_hours'].iloc[-1]
            
            def model(t, *params):
                a1 = params[0]
                result = a1
                for k in range(2, max_k + 2):
                    result += params[k - 1] * (tref-t) ** (1 / k)
                return result

            t_data = tdf['time_hours'].values
            y_data = tdf['altitude'].values

            initial_guess = np.ones(max_k + 1)
                
            params, covariance = curve_fit(model, t_data, y_data, p0=initial_guess)

            def predict_times_for_altitudes(h_targets, params, t_bounds):
                predicted_times = []
                for h_target in h_targets:
                    def func(t):
                        return model(t, *params) - h_target

                    f_a = func(t_bounds[0])
                    f_b = func(t_bounds[1])

                    if f_a * f_b > 0:
                        # No sign change: root not guaranteed in the interval
                        predicted_times.append(np.nan)
                        continue

                    try:
                        result = root_scalar(func, bracket=t_bounds, method='brentq')
                        if result.converged:
                            predicted_times.append(result.root)
                        else:
                            predicted_times.append(np.nan)
                    except ValueError:
                        predicted_times.append(np.nan)
                return predicted_times
            
            
            target_altitudes = np.linspace(200, 80,25)
            t_bounds = (t_data[0], t_data[-1])
            times_to_targets = predict_times_for_altitudes(target_altitudes, params, t_bounds)
            times_to_targets = np.array(times_to_targets)

            if np.count_nonzero(np.isnan(times_to_targets)) == 1 and np.isnan(times_to_targets[-1]):
                times_to_targets[-1] = tdf['time_hours'].iloc[-1]
            elif np.count_nonzero(np.isnan(times_to_targets)) > 1:
                invalid_dataset.append(id)
                continue

            final_df = pd.DataFrame(columns=['edt', 'altitude'])
            t0 = tdf['EPOCH_DATE_TIME'].min()
            for x,y in zip(target_altitudes, times_to_targets):
                final_df.loc[len(final_df)] = [t0 + pd.to_timedelta(y, unit='h'), x]
            
            final_df.sort_values(by=['edt'], inplace=True)
            final_df['alt_diff'] = final_df['altitude'].diff()
            if (final_df['alt_diff'].iloc[1:] > 0).any():
                # print("Altitude is NOT strictly decreasing: there is at least one upward jump.")
                # # If you want to see exactly where:
                bad_idxs = df.index[df['alt_diff'] > 0].tolist()
                print("↑ Violations at row indices:", bad_idxs)
                print("↑ Violations at row values:", df.loc[bad_idxs, 'alt_diff'])
                invalid_dataset.append(id)
                continue
            else:
                final_df = final_df.drop('alt_diff', axis=1)
                final_df.to_csv(f'datasets/{id}.csv', index=False, date_format='%Y-%m-%d %H:%M:%S.%f%z')
        except Exception as e:
            print(e)
            print(id)
            invalid_dataset.append(id)

    return invalid_dataset

In [ ]:
invalid_dataset = generate_dataset()

In [ ]:
len(invalid_dataset)

In [ ]:
v = 0
m = 0
z = 0
for id, tip in zip(valid_decayed_list[z:], valid_decayed_tips[z:]):
    tles = read_tle_file(f'objects/{id}.txt')
    df = generate_df_altitude_with_tip(tles, tip)
    df = df.reset_index(drop=True)
    if len(df) <= 3:
        m += 1
    else:
        v += 1
    print(f"{v+m} / {len(valid_decayed_list)}")

In [ ]:
# INPUT_SEQ_LEN = 15
# OUTPUT_SEQ_LEN = 10
# path = 'datasets/60.csv'
# df = pd.read_csv(path)

# df['edt'] = pd.to_datetime(df['edt'], format='%Y-%m-%d %H:%M:%S.%f')
# df['hours'] = (df['edt'] - df['edt'].iloc[0]).dt.total_seconds() / 3600.0
# hours = df['hours'].values.astype(np.float32)

# scaler = MinMaxScaler()
# norm_hours = scaler.fit_transform(hours.reshape(-1,1)).flatten()

# encoder_inputs, decoder_inputs, decoder_targets = [], [], []

# encoder_inputs.append(norm_hours[:INPUT_SEQ_LEN].reshape(-1, 1))
# di = norm_hours[INPUT_SEQ_LEN - 1:-1].copy()
# di[0] = 0.0
# decoder_inputs.append(di.reshape(-1, 1))
# decoder_targets.append(norm_hours[INPUT_SEQ_LEN:].reshape(-1, 1))

In [ ]:
# Constants
INPUT_SEQ_LEN = 15
OUTPUT_SEQ_LEN = 10

# Data generator with decoder inputs
class SequenceGenerator(Sequence):
    def __init__(self, file_paths, batch_size=32):
        self.file_paths = file_paths
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.file_paths) / self.batch_size))

    def __getitem__(self, idx):
        batch_paths = self.file_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        encoder_inputs, decoder_inputs, decoder_targets = [], [], []

        for path in batch_paths:
            df = pd.read_csv(path)
            if len(df) != INPUT_SEQ_LEN + OUTPUT_SEQ_LEN:
                continue

            df['edt'] = pd.to_datetime(df['edt'], format='%Y-%m-%d %H:%M:%S.%f%z')
            df['hours'] = (df['edt'] - df['edt'].iloc[0]).dt.total_seconds() / 3600.0
            hours = df['hours'].values.astype(np.float32)

            scaler = MinMaxScaler()
            norm_hours = scaler.fit_transform(hours.reshape(-1,1)).flatten()

            encoder_inputs.append(norm_hours[:INPUT_SEQ_LEN].reshape(-1, 1))
            decoder_inputs.append(norm_hours[INPUT_SEQ_LEN - 1:-1].reshape(-1, 1))
            decoder_targets.append(norm_hours[INPUT_SEQ_LEN:].reshape(-1, 1))


            # encoder_inputs.append(hours[:INPUT_SEQ_LEN].reshape(-1, 1))
            # decoder_inputs.append(hours[INPUT_SEQ_LEN - 1:-1].reshape(-1, 1))
            # decoder_targets.append(hours[INPUT_SEQ_LEN:].reshape(-1, 1))

        return [np.array(encoder_inputs), np.array(decoder_inputs)], np.array(decoder_targets)

In [ ]:
def build_seq2seq_gru(latent_dim=64):
    encoder_inputs = Input(shape=(INPUT_SEQ_LEN, 1), name='encoder_inputs')
    encoder_gru1 = GRU(latent_dim, return_sequences=True, name='encoder_gru_1')
    encoder_seq1 = encoder_gru1(encoder_inputs)

    encoder_gru2 = GRU(latent_dim, return_sequences=True, name='encoder_gru_2')
    encoder_seq2 = encoder_gru2(encoder_seq1)

    encoder_gru3 = GRU(latent_dim, return_sequences=True, name='encoder_gru_3')
    encoder_seq3 = encoder_gru3(encoder_seq2)

    encoder_gru4 = GRU(latent_dim, return_sequences=True, name='encoder_gru_4')
    encoder_seq4 = encoder_gru4(encoder_seq3)

    # encoder_gru5 = GRU(latent_dim, return_sequences=True, name='encoder_gru_5')
    # encoder_seq5 = encoder_gru5(encoder_seq4)

    # encoder_gru6 = GRU(latent_dim, return_sequences=True, name='encoder_gru_6')
    # encoder_seq6 = encoder_gru6(encoder_seq5)

    # encoder_gru7 = GRU(latent_dim, return_sequences=True, name='encoder_gru_7')
    # encoder_seq7 = encoder_gru7(encoder_seq6)

    encoder_gru8 = GRU(latent_dim, return_state=True, name='encoder_gru_8')
    _, state_h = encoder_gru8(encoder_seq4)

    decoder_inputs = Input(shape=(OUTPUT_SEQ_LEN, 1), name='decoder_inputs')
    decoder_gru1 = GRU(latent_dim, return_sequences=True, name='decoder_gru_1')
    decoder_seq1 = decoder_gru1(decoder_inputs, initial_state=state_h)

    decoder_gru2 = GRU(latent_dim, return_sequences=True, name='decoder_gru_2')
    decoder_seq2 = decoder_gru2(decoder_seq1)

    decoder_gru3 = GRU(latent_dim, return_sequences=True, name='decoder_gru_3')
    decoder_seq3 = decoder_gru3(decoder_seq2)

    decoder_gru4 = GRU(latent_dim, return_sequences=True, name='decoder_gru_4')
    decoder_seq4 = decoder_gru4(decoder_seq3)

    decoder_gru5 = GRU(latent_dim, return_sequences=True, name='decoder_gru_5')
    decoder_seq5 = decoder_gru5(decoder_seq4)

    # decoder_gru6 = GRU(latent_dim, return_sequences=True, name='decoder_gru_6')
    # decoder_seq6 = decoder_gru6(decoder_seq5)

    # decoder_gru7 = GRU(latent_dim, return_sequences=True, name='decoder_gru_7')
    # decoder_seq7 = decoder_gru7(decoder_seq6)

    # decoder_gru8 = GRU(latent_dim, return_sequences=True, name='decoder_gru_8')
    # decoder_seq8 = decoder_gru8(decoder_seq7)

    decoder_dense = TimeDistributed(Dense(1, name='decoder_dense_node_1'), name='decoder_dense_dist_1')
    decoder_outputs = decoder_dense(decoder_seq5)

    rmse = tf.keras.metrics.RootMeanSquaredError()
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse', metrics=[rmse, 'mae'])
    return model

In [ ]:
directory = 'datasets'
csv_files = glob.glob(os.path.join(directory, '*.csv'))
train_files, test_files = train_test_split(
        csv_files,
        test_size=0.1,    
        random_state=42,
)
batch_size = 32
epochs = 2000

train_gen = SequenceGenerator(train_files, batch_size=32)
test_gen = SequenceGenerator(test_files, batch_size=32)

print(f"Training: {len(train_files)} files; Validation: {len(test_files)} files")

checkpoint_path = "case_D/layers_5/neurons_100/gru/training_1_gru/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                    patience = 20, restore_best_weights = False)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    monitor="val_loss",
    mode="min",
    save_weights_only=True,
    save_best_only=True,
    verbose=1
)

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)

gru_model = build_seq2seq_gru(100)
gru_model.summary()

In [ ]:
history = gru_model.fit(
    train_gen,
    validation_data = test_gen,
    epochs=epochs,
    callbacks=[ checkpoint, lr_scheduler, early_stop]
)
gru_model.save('case_D/layers_5/neurons_300/gru/seq2seq_altitude_gru', save_format='tf')

In [ ]:
id = 38086
path = f"datasets/{id}.csv"
df = pd.read_csv(path)

df['edt'] = pd.to_datetime(df['edt'], format='%Y-%m-%d %H:%M:%S.%f%z')
df['hours'] = (df['edt'] - df['edt'].iloc[0]).dt.total_seconds() / 3600.0
hours = df['hours'].values.astype(np.float32)

scaler = MinMaxScaler()
norm_hours = scaler.fit_transform(hours.reshape(-1,1)).flatten()

encoder_input = norm_hours[:INPUT_SEQ_LEN].reshape(1, INPUT_SEQ_LEN, 1)

In [ ]:
plt.figure(figsize=(8, 5))
plt.scatter(df['hours'], df['altitude'],
            color='blue', label='Data points')
plt.show()

In [ ]:
gru_model.load_weights(checkpoint_path)

In [ ]:
decoder_input = np.zeros((1, OUTPUT_SEQ_LEN, 1))
decoder_input[0, 0, 0] = encoder_input[0, -1, 0]

# Placeholder for predictions
predictions = []

for i in range(OUTPUT_SEQ_LEN):
    # Predict the next value
    output = gru_model.predict([encoder_input, decoder_input], verbose=0)
    next_value = output[0, i, 0]
    predictions.append(next_value)
    
    # Update the decoder input for the next time step
    if i + 1 < OUTPUT_SEQ_LEN:
        decoder_input[0, i + 1, 0] = next_value

In [ ]:
predictions = np.array(predictions)
predicted_values = scaler.inverse_transform(predictions.reshape(-1,1))
true_future = hours[INPUT_SEQ_LEN:] 
predictions_hr = predicted_values.flatten()
rel_err_percent = np.full_like(true_future, np.nan, dtype=np.float32)
mask_nonzero = true_future != 0
rel_err_percent[mask_nonzero] = (
    (predictions_hr[mask_nonzero] - true_future[mask_nonzero])
    / true_future[mask_nonzero]
) * 100.0
abs_rel_err_percent = np.abs(rel_err_percent)
mean_rel_err_percent      = np.nanmean(rel_err_percent) 
abs_error = np.abs(predictions_hr - true_future)  
MAE_hours = abs_error.mean()

In [ ]:
plt.figure(figsize=(8, 5))
plt.scatter(df['hours'], df['altitude'],
            color='blue', label='Data points')

plt.plot(predicted_values, df['altitude'].iloc[INPUT_SEQ_LEN:], color='red', linestyle='--', marker='o',
         label='predicted')
plt.plot([], [], ' ', label=f'Absolute Error (hours): {MAE_hours:.2f}')
plt.plot([], [], ' ', label=f'Relative Error (%): {mean_rel_err_percent:.2f}')
plt.xlabel('elapsed time (hours) from altitude = 200 km')
plt.ylabel('altitude (km)')
plt.title(f'Object {id} 5 layer 100 n')
plt.legend()
plt.grid(True)
plt.show()